In [3]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
from PIL import Image # converting images into arrays
import requests 
from bs4 import BeautifulSoup 
from pandas.io.json import json_normalize
import csv

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.style.use('ggplot') # optional: for ggplot-like style

# check for latest version of Matplotlib
print ('Matplotlib version: ', mpl.__version__) # >= 2.0.0

# install wordcloud
!conda install -c conda-forge wordcloud==1.4.1 --yes

# import package and its set of stopwords
from wordcloud import WordCloud, STOPWORDS

print ('Wordcloud is installed and imported!')

Matplotlib version:  3.1.3
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
failed
Wordcloud is installed and imported!



Examining wordcloud==1.4.1: 100%|##########| 2/2 [00:00<00:00,  6.74it/s]
                                                                         

Examining conflict for python wordcloud:   0%|          | 0/2 [00:00<?, ?it/s]
                                                                              

UnsatisfiableError: The following specifications were found
to be incompatible with the existing python installation in your environment:

Specifications:

  - wordcloud==1.4.1 -> python[version='2.7.*|3.5.*|3.6.*']

Your python: python=3.7

If python is on the left-most side of the chain, that's the version you've asked for.
When python appears to the right, that indicates that the thing on the left is somehow
not available for the python version you are constrained to. Note that conda will not
change your python version to a different minor version unless you explicitly specify
that.





In [4]:

# Read the CSV data file from the object storage into a pandas DataFrame

import pandas as pd
NYC_CUISINE=pd.read_csv("BON3_NYC_CUISINE.csv")
print(NYC_CUISINE)
#NYC_CUISINE.drop(NYC_CUISINE.columns[[3,4,5,6,7]], axis=1,inplace=True) 
NYC_CUISINE.head()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE
0,40511702,NOTARO RESTAURANT,MANHATTAN,635,SECOND AVENUE,10016.0,2126863400,Italian,6/15/2015,Violations were cited in the following area(s).,02B,Hot food item not held at or above 140Âº F.,Critical,30.0,NaN,NaN,8/28/2017,Cycle Inspection / Initial Inspection
1,40511702,NOTARO RESTAURANT,MANHATTAN,635,SECOND AVENUE,10016.0,2126863400,Italian,11/25/2014,Violations were cited in the following area(s).,20F,Current letter grade card not posted.,Not Critical,NaN,NaN,NaN,8/28/2017,Administrative Miscellaneous / Initial Inspection
2,50046354,VITE BAR,QUEENS,2507,BROADWAY,11106.0,3478134702,Italian,10/3/2016,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,2.0,NaN,NaN,8/28/2017,Pre-permit (Operational) / Initial Inspection
3,50061389,TACK'S CHINESE TAKE OUT,STATEN ISLAND,11C,HOLDEN BLVD,10314.0,7189839854,Chinese,5/17/2017,Violations were cited in the following area(s).,02G,Cold food item held above 41Âº F (smoked fish ...,Critical,46.0,NaN,NaN,8/28/2017,Pre-permit (Operational) / Initial Inspection
4,41516263,NO QUARTER,BROOKLYN,8015,5 AVENUE,11209.0,7187019180,American,3/30/2017,Violations were cited in the following area(s).,04M,Live roaches present in facility's food and/or...,Critical,18.0,NaN,NaN,8/28/2017,Cycle Inspection / Initial Inspection


In [5]:
NYC_CUISINE.shape

(399918, 18)

In [2]:
print(NYC_CUISINE.BORO.unique())

['MANHATTAN' 'QUEENS' 'STATEN ISLAND' 'BROOKLYN' 'BRONX' 'Missing']


In [4]:
CUISINE_WC = NYC_CUISINE[['CUISINE DESCRIPTION']]
CUISINE_WC

,CUISINE DESCRIPTION
0,Italian
1,Italian
2,Italian
3,Chinese
4,American
...,...
399913,Jewish/Kosher
399914,American
399915,Japanese
399916,American


In [7]:
Queens_data = NYC_CUISINE[NYC_CUISINE['BORO'] == 'QUEENS'].reset_index(drop=True)
Queens_data.head()

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE
0,50046354,VITE BAR,QUEENS,2507,BROADWAY,11106.0,3478134702,Italian,10/3/2016,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,2.0,NaN,NaN,8/28/2017,Pre-permit (Operational) / Initial Inspection
1,50015855,KABAB HOUSE NYC,QUEENS,4339,MAIN ST,11355.0,9172852796,Pakistani,3/3/2015,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,19.0,NaN,NaN,8/28/2017,Pre-permit (Operational) / Compliance Inspection
2,41696945,GOLDEN KRUST CARIBBEAN BAKERY & GRILL,QUEENS,13858,FRANCIS LEWIS BOULEVARD,11422.0,7183412011,Caribbean,10/14/2015,Violations were cited in the following area(s).,04N,Filth flies or food/refuse/sewage-associated (...,Critical,24.0,NaN,NaN,8/28/2017,Cycle Inspection / Initial Inspection
3,41690659,NEW PANDA GARDEN,QUEENS,5917,ROOSEVELT AVENUE,11377.0,7184241038,Chinese,8/28/2014,Violations were cited in the following area(s).,04N,Filth flies or food/refuse/sewage-associated (...,Critical,28.0,NaN,NaN,8/28/2017,Cycle Inspection / Initial Inspection
4,50018449,SPRING FISH VILLAGE,QUEENS,13621,41ST AVE,11355.0,7189393633,Chinese,3/21/2016,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,19.0,NaN,NaN,8/28/2017,Cycle Inspection / Initial Inspection


In [10]:
QN_CUISINE_WC = Queens_data[['CUISINE DESCRIPTION']]
QN_CUISINE_WC.head()

,CUISINE DESCRIPTION
0,Italian
1,Pakistani
2,Caribbean
3,Chinese
4,Chinese


In [2]:
Cityhall = NYC_CUISINE[NYC_CUISINE['ZIPCODE'] == '10007'].reset_index(drop=True)
Cityhall.head()

NameError: name 'NYC_CUISINE' is not defined